# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
client.list_database_names()

['First_try', 'admin', 'config', 'ironhack', 'local']

In [2]:
db = client.get_database("ironhack")
db.list_collection_names()

['companies']

In [3]:
c = db.get_collection("companies")
#c.find_one()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
c_name = "Babelgum"
List_Babelgum = list(c.find({"name":c_name}))
#List_Babelgum #solo hay un resultado.

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [5]:
# Your Code #number_of_employees
more_5000 = {"$gt": 5000}
emplo_5000 = list(c.find({"number_of_employees":more_5000}).sort("number_of_employees",-1).limit(20))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [6]:
#founded_year
proy_name = {"name":1,"_id":0}
more_2000 ={"founded_year" : {"$gt":2000}}
less_2005 ={"founded_year" : {"$lt":2005}}
list(c.find({"$and":[more_2000,less_2005]},proy_name))[0:5]

[{'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'StumbleUpon'},
 {'name': 'Gizmoz'},
 {'name': 'Plaxo'}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [7]:
proy_name_ipo = {"name":1,"_id":0,"ipo":1}
va_100 = {"ipo.valuation_amount": {"$gt":100000000} }
found_2010 ={"founded_year" : {"$lt":2010}}
list(c.find({"$and":[va_100,found_2010]},proy_name_ipo))[0:2]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [8]:
emplo_1000 = {"number_of_employees":{"$lt":1000}}
less_2005 ={"founded_year" : {"$lt":2005}}
lista = list(c.find({"$and":[emplo_1000,less_2005]}).sort("number_of_employees",-1).limit(10))

### 6. All the companies that don't include the `partners` field.

In [9]:
part = {"partners":{"$exists": False}}
list(c.find(part))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [10]:
lista = list(c.find({"category_code" : {"$type": "null"}}))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [11]:
proy_name_num = {"name":1,"_id":0,"number_of_employees":1}
emplo_1000 = {"number_of_employees":{"$lt":1000}}
emplo_l_100 = {"number_of_employees":{"$gte":100}}
list(c.find({"$and":[emplo_1000,emplo_l_100]},proy_name_num))[0:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [12]:
#list(c.find().sort("ipo.valuation_amount",-1))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [13]:
#list(c.find().sort("number_of_employees",-1).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [14]:
found_sem = {"founded_month":{"$gte":6}}
#list(c.find(found_sem).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [15]:
found_200 = {"founded_year":{"$lt":2000}}
ac_10 = {"acquisitions.price_amount":{"$gt":10000000}}
#list(c.find({"$and":[found_200, ac_10]}))[0]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [16]:
proy_ac = {"_id":0, "acquisition":1, "name":1}
after_2010 =  {"acquisitions.acquired_year": {"$gt":2010}}
list(c.find(after_2010,proy_ac).sort("acquisition",1))[0:5]

[{'name': 'Facebook', 'acquisition': None},
 {'name': 'Twitter', 'acquisition': None},
 {'name': 'eBay', 'acquisition': None},
 {'name': 'CBS', 'acquisition': None},
 {'name': 'Cisco', 'acquisition': None}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [17]:
proy_name_year = {"_id":0, "name":1}
list(c.find({},proy_name_year).sort("founded_year",1))[0:3]

[{'name': 'Flektor'}, {'name': 'Lala'}, {'name': 'SpinVox'}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [18]:
f_month = {"founded_month":1}
f_day = {"founded_day":{"$lte":7}}
#list(c.find({"$and":[f_month,f_day]}).sort("acquisitions.price_amount",-1).limit(10))

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [19]:
#category_code': 'web',
cat_Web ={'category_code': 'web'}
emp_4000 = {'number_of_employees':{"$gt":4000}}
#list(c.find({"$and":[cat_Web,emp_4000]}).sort("number_of_employees",1))[0:3]


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [20]:
#acquisition.price_currency_code   price_amount
ac_pr = {"acquisition.price_amount":{"$gt":10000000}}
ac_curr = {"acquisition.price_currency_code":"EUR" }
#list(c.find({"$and" : [ac_pr,ac_curr]}))

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [21]:
pro_first = {"_id":0,"name":1,"acquisition":1}
first_tri = {"founded_month":{"$lte":3}}
list(c.find(first_tri,pro_first).limit(10))[0:1]

[{'name': 'Facebook', 'acquisition': None}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [22]:
found_before = {"founded_year":{"$lte":2010}}
found_after = {"founded_year":{"$gte":2000}}
ac_year = {"acquisitions.acquired_year":{"$gte":2011}}

#list(c.find({"$and":[found_before,found_after,ac_year]}))

### 20. All the companies that have been 'deadpooled' after the third year.

In [23]:
### list(c.find({"$where": "(deadpooled_year - founded_year > 3)"} ))
# lo he intentado de muchas maneras y soy incapaz de averiguar cómo hacerlo...

len(list(c.find({"$where": "(this.deadpooled_year - this.founded_year > 3)"} )))

#Ras ha explicado que hacía falta el this. y funciona perfectamente... :

655